In [3]:
import awkward as ak
import numpy as np
import hist as hs
from coffea import processor, hist as chs
from coffea.nanoevents.methods import vector, candidate
from numba import jit

from coffea.nanoevents import BaseSchema
import mplhep as hep
import matplotlib.pyplot as plt
plt.style.use(hep.style.CMS)

In [36]:
class LLP_ntuple_processor(processor.ProcessorABC):
    """
    This class is used to process the ntuples created by the LLP ntuple producer.
    """

    def acc_cut(self, events):

        cut = (events.gLLP_csc == 1)
        return events[ak.any(cut, axis = -1)]

    def csc_eff_cut(self, events):
        
        cut = (events.nCscRechitClusters >= 1)
        return events[ak.any(cut, axis = -1)]

    def muon_veto(self, events):
        
        cut = (events.cscRechitClusterMuonVetoPt < 20)
        return events[ak.any(cut, axis = -1)]

    def jet_cut(self, events):
        
        cut = (events.cscRechitClusterJetVetoPt < 10)
        return events[ak.any(cut, axis = -1)]

    def time_cut(self, events):

        cut = ((events.cscRechitClusterTimeWeighted <= 12.5)&
               (events.cscRechitClusterTimeWeighted >= -5  ))
        return events[ak.any(cut, axis = -1)]

    def ME11_veto(self, events):

        cut = ((events.cscRechitClusterNRechitChamberPlus11 <= 0)&
               (events.cscRechitClusterNRechitChamberMinus11 <= 0))
        return events[ak.any(cut, axis = -1)]


    def ME12_veto(self, events):

        cut = ((events.cscRechitClusterNRechitChamberPlus12 <= 0)&
               (events.cscRechitClusterNRechitChamberMinus12 <= 0))
        return events[ak.any(cut, axis = -1)]


    def re12_cut(self, events):
        
        cut = events.cscRechitCluster_match_RE12_0p4 == 0
        return events[ak.any(cut, axis = -1)]

    def mb1_cut(self, events):
        
        cut = events.cscRechitCluster_match_MB1Seg_0p4 == 0
        return events[ak.any(cut, axis = -1)]

    def rb1_cut(self, events):
        
        cut = events.cscRechitCluster_match_RB1_0p4 == 0
        return events[ak.any(cut, axis = -1)]

    def eta_cut(self, events):
        
        cut = abs(events.gLLP_eta) < 2 
        return events[ak.any(cut, axis = -1)]

    def time_spread_cut(self, events):
    
        cut = events.cscRechitClusterTimeSpreadWeightedAll <= 20
        return events[ak.any(cut, axis = -1)]

    def cut_based(self, events):
    
        cut = (((events.cscRechitClusterNStation10 > 1) & (abs(events.cscRechitClusterEta) < 1.9))|
               ((events.cscRechitClusterNStation11 == 1) & (events.cscRechitClusterAvgStation10 == 4) & (abs(events.cscRechitClusterEta) < 1.8))|
               ((events.cscRechitClusterNStation11 == 1) & (events.cscRechitClusterAvgStation10 == 3) & (abs(events.cscRechitClusterEta) < 1.6))|
               ((events.cscRechitClusterNStation11 == 1) & (events.cscRechitClusterAvgStation10 == 2) & (abs(events.cscRechitClusterEta) < 1.6))|
               ((events.cscRechitClusterNStation11 == 1) & (events.cscRechitClusterAvgStation10 == 1) & (abs(events.cscRechitClusterEta) < 1.1))
            )
        return events[ak.any(cut, axis = -1)]

    def dphi_cut(self, events):
    
        cut = abs(events.cscRechitClusterMetEENoise_dPhi) < 0.75
        return events[ak.any(cut, axis = -1)]

    def nrechits_cut(self, events):

        cut = events.cscRechitClusterSize > 130
        return events[ak.any(cut, axis = -1)]


    def process(self, events):

        dataset = events.metadata['dataset']
        sumw = ak.sum(events.weight)

        out = {
            dataset: {
                "entries": len(events),
                "sumw": sumw,
            }
        }
         
        #cuts = [self.acc_cut, self.csc_eff_cut, self.muon_veto, self.jet_cut, self.time_cut, self.ME11_veto, self.ME12_veto, self.re12_cut, self.mb1_cut, self.rb1_cut, self.eta_cut, self.time_spread_cut, self.cut_based, self.dphi_cut, self.nrechits_cut]

        '''last = events
        out[dataset]['total'] = len(last)
        for cut in cuts:
            last = cut(last)
            out[dataset][cut.__name__] = len(last)'''

        temp = self.acc_cut(events)
        #print(temp)
        out[dataset]['total'] = len(temp)

        return out   
    
    def postprocess(self, accumulator):
        return accumulator

In [37]:
fileset = {}
fileset['BToPhiK'] = ['root://cmsxrootd.fnal.gov//store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/MC_Fall18/v2/v3/normalized/BToKPhi_MuonGenFilter_mPhi1p0_ctau1000_1pb_weighted.root',]

out = processor.run_uproot_job(
    fileset,
    treename="MuonSystem",
    processor_instance=LLP_ntuple_processor(),
    executor=processor.futures_executor,
    executor_args={"schema": BaseSchema, "workers": 1},
    maxchunks = 1
)

Processing   0% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/1 [ 0:00:00 < -:--:-- | ?  chunk/s ]
Merging (local)   0% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/0 [ 0:00:00 < -:--:-- | ? merges/s ]

... False], dtRechitClusterMetEENoise_dPhi: [1.02], weight: 1.21e-06}


concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/uscms_data/d3/aaportel/mambaforge/envs/analysis-env/lib/python3.9/site-packages/coffea/processor/executor.py", line 1635, in _work_function
    out = processor_instance.process(events)
  File "/tmp/ipykernel_15223/1817244852.py", line 114, in process
    out[dataset]['total'] = len(temp)
TypeError: object of type 'NanoEventsRecord' has no len()

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/uscms_data/d3/aaportel/mambaforge/envs/analysis-env/lib/python3.9/concurrent/futures/process.py", line 246, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
  File "/uscms_data/d3/aaportel/mambaforge/envs/analysis-env/lib/python3.9/site-packages/coffea/processor/executor.py", line 220, in __call__
    out = self.function(*args, **kwargs)
  File "/uscms_data/d3/aaportel/mambaforge/envs/analysis-env/lib/python3.9/s

Exception: Failed processing file: WorkItem(dataset='BToPhiK', filename='root://cmsxrootd.fnal.gov//store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/MC_Fall18/v2/v3/normalized/BToKPhi_MuonGenFilter_mPhi1p0_ctau1000_1pb_weighted.root', treename='MuonSystem', entrystart=0, entrystop=102974, fileuuid=b'\xec\xaf\xa3\x90\xf75\x11\xec\xab\x03f\n\x03\n\xbe\xef', usermeta={})

In [18]:
out

{'BToPhiK': {'entries': 823789, 'sumw': 0.9999997}}

In [20]:
out

{'BToPhiK': {'entries': 102974, 'sumw': 0.12500042}}